In [ ]:
from __future__ import print_function, division, absolute_import

%load_ext autoreload
%autoreload 2

import safeopt
import GPy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets # Widget definitions
from IPython import display # Used to display widgets in the notebook

## Define a kernel and function

Here we define a kernel. The function is drawn at random from the GP and is corrupted my Gaussian noise

In [ ]:
# Measurement noise
noise_var = 0.05 ** 2

# Set fixed Gaussian measurement noise
likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)

# Bounds on the inputs variable
bounds = [(-10., 10.), (-1, 1)]

# Define Kernel
# works on the first column of X, index=0
k1 = GPy.kern.RBF(input_dim=1, variance=2., lengthscale=1.0, active_dims=[0])
# works on the second column of X, index=1
# k2 = GPy.kern.Linear(input_dim=1, active_dims=[1])
k2 = GPy.kern.RBF(input_dim=1, variance=2., lengthscale=1.0, active_dims=[1])
kernel = k1 * k2

# set of parameters
parameter_set = safeopt.linearly_spaced_combinations([bounds[0]], 1000)

# Initial safe point
x0 = np.array([[0]])

# Generate function with safe initial point at x=0
def sample_safe_fun():
    while True:
        k1_fun = safeopt.sample_gp_function(k1, [bounds[0]], noise_var, 50)    
        if k1_fun(0, noise=False) > 0.5:
            break
    def fun(x, context, noise=True):
        return k1_fun(x, noise=noise) + (5 * context)
    return fun

## Interactive run of the algorithm

In [ ]:
button_add = widgets.Button(description='Add sample', tooltip='Sample a new data point for the optimization')
button_reset = widgets.Button(description='Reset', tooltip='Restart the safe optimization algorithm')
button_new = widgets.Button(description='New function', tooltip='Add a new function and reset')
slider_safety = widgets.FloatSlider(description='Safety threshold', value=0, min=-4, max=4, step=0.1)
slider_context = widgets.FloatSlider(description='         Context', value=0, min=-1, max=1, step=0.05)

ms = 20
mew = 7
lw = 5

context = np.array([[0]])

def plot_gp():
    gp_opt.plot(figure=plt.figure(figsize=(15, 8)), lw=lw)
    
    # Plot last point red
    if gp_opt.gp is not None:
        plt.plot(gp_opt.gp.X[-1, 0], gp_opt.gp.Y[-1, :], 'rx', ms=ms, mew=mew, label='Last Point')
        
    # Plot safe line
    plt.plot([bounds[0][0], bounds[0][1]], [gp_opt.fmin, gp_opt.fmin], 'k--', label='Minimum', lw=lw)
        
    plt.plot(true_values[0], true_values[1],
             alpha=0.15, label='True function', lw=lw)
    
    plt.ylim([-4, 4])
    plt.legend(loc=3)
    
    # Ensure we only get one plot
    display.clear_output(wait=True)

def new_sample(b=None):
    """Draw a new gp sample"""
    gp_opt.optimize(context=context)
    plot_gp()
button_add.on_click(new_sample)

def reset(b=None):
    """Reset the GP-UCB algorithm"""
    global gp_opt
    gp = GPy.core.GP(np.hstack((x0, [[0]])), fun(x0, 0), kernel, likelihood)
    gp_opt = safeopt.SafeOpt(fun, gp, parameter_set, 0., num_contexts=1)
    plot_gp()
button_reset.on_click(reset)

def new_fun(b=None):
    """Draw a new function from the GP"""
    global fun
    fun = sample_safe_fun()
    get_true_fun()
    reset(b)
button_new.on_click(new_fun)

def get_true_fun():
    global true_values
    inp = np.linspace(bounds[0][0], bounds[0][1], 200)[:, None]
    true_values = (inp, fun(inp, context, noise=False))

def change_safety(b=None):
    if b == 'value':
        gp_opt.fmin = slider_safety.get_state()['value']
        plot_gp()
slider_safety.on_trait_change(change_safety)

def change_context(b=None):
    if b == 'value':
        global context
        context = slider_context.get_state()['value']
        get_true_fun()
        gp_opt.change_context(context)
        plot_gp()
slider_context.on_trait_change(change_context)

display.display(button_add, button_new, button_reset, slider_safety, slider_context)
new_fun()